# Capstone Project - Open a Musical Theme Café in London 

## Table of contents
* [Introduction](#introduction)

## Introduction <a name="introduction"></a>

In this project we will try to find an optimal location for a cafe. Specifically, this report will be targeted to stakeholders interested in opening an **Musical Theme Cafe** in **London**, United Kingdom.

Since there are lots of cafes in London we will try to detect **locations that are not already crowded with cafes**. As this cafe is musical themed,we would also prefer locations **as close to London's West End center as possible**, where crowded with all the famous theaters in the UK.

We will use our data science powers to generate a few most promissing neighborhoods based on this criteria. Advantages of each area will then be clearly expressed so that best possible final location can be chosen by stakeholders.

## Data <a name="data"><a/>

Based on definition of our problem, factors that will influence our decission are:
* number of existing cafes in the neighborhood 
* distance of neighborhood from West End center (theater center)

We decided to use regularly spaced grid of locations, centered around city center, to define our neighborhoods.

Following data sources will be needed to extract/generate the required information:
* centers of candidate areas will be generated algorithmically and approximate addresses of centers of those areas will be obtained using **geolocator.geocode**
* number of cafes and location in every neighborhood will be obtained using **Foursquare API**
* coordinate of West End center will be obtained using **geolocator.geocode** of well known London location (Lercester Square)

### Neighborhood Candidates

Let's create latitude & longitude coordinates for centroids of our candidate neighborhoods. We will create a grid of cells covering our area of interest which is aprox. 4x4 killometers centered around West End center.

Let's first find the latitude & longitude of West End center, using specific, well known address and geolocator.

In [1]:
# import liraries
import numpy as np 
import pandas as pd
import json 
import requests
from pandas.io.json import json_normalize
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
import folium

In [2]:
from geopy.geocoders import Nominatim 
def get_coordinates (address):
    try:
        geolocator = Nominatim(user_agent="coordinates_explorer", timeout=200)
        location = geolocator.geocode(address)
        lat = location.latitude
        lon = location.longitude
        return [lat, lon]
    except:
        return [None, None]
# get coordinates of Leicester Square and name it theater_certer    
address = 'Leicester Square, United Kingdom'
theater_center = get_coordinates(address)
print('Coordinate of {}: {}'.format(address, theater_center))

Coordinate of Leicester Square, United Kingdom: [51.5105992, -0.1308565]


Now let's create a grid of area candidates, equaly spaced, centered around city center and within ~2km from Leicester Square. Our neighborhoods will be defined as circular areas with a radius of 100 meters, so our neighborhood centers will be 200 meters apart.

To accurately calculate distances we need to create our grid of locations in Cartesian 2D coordinate system which allows us to calculate distances in meters (not in latitude/longitude degrees). Then we'll project those coordinates back to latitude/longitude degrees to be shown on Folium map. So let's create functions to convert between WGS84 spherical coordinate system (latitude/longitude degrees) and UTM Cartesian coordinate system (X/Y coordinates in  meters).

In [3]:
import shapely.geometry

import pyproj
from pyproj import Transformer

import math

def lonlat_to_xy(lon, lat):
    proj_latlon = pyproj.Proj(proj='latlong',datum='WGS84')
    proj_xy = pyproj.Proj(proj="utm", zone=31, datum='WGS84')
    transformer = Transformer.from_proj(proj_latlon, proj_xy)
    xy = transformer.transform(lon, lat)
    return xy[0], xy[1]

def xy_to_lonlat(x, y):
    proj_latlon = pyproj.Proj(proj='latlong',datum='WGS84')
    proj_xy = pyproj.Proj(proj="utm", zone=31, datum='WGS84')
    transformer = Transformer.from_proj(proj_xy, proj_latlon)
    lonlat = transformer.transform(x, y)
    return lonlat[0], lonlat[1]

def calc_xy_distance(x1, y1, x2, y2):
    dx = x2 - x1
    dy = y2 - y1
    return math.sqrt(dx*dx + dy*dy)

print('Coordinate transformation check')
print('-------------------------------')
print('Theater center longitude={}, latitude={}'.format(theater_center[1], theater_center[0]))
x, y = lonlat_to_xy(theater_center[1], theater_center[0])
print('Theater center UTM X={}, Y={}'.format(x, y))
lo, la = xy_to_lonlat(x, y)
print('Theater center longitude={}, latitude={}'.format(lo, la))


Coordinate transformation check
-------------------------------
Theater center longitude=-0.1308565, latitude=51.5105992
Theater center UTM X=282753.38803952804, Y=5711255.921690965
Theater center longitude=-0.13085649999999918, latitude=51.5105992


Let's create a **hexagonal grid of cells**: we offset every other row, and adjust vertical row spacing so that **every cell center is equally distant from all it's neighbors**.

In [4]:
theater_center_x, theater_center_y = lonlat_to_xy(theater_center[1], theater_center[0]) # City center in Cartesian coordinates

k = math.sqrt(3) / 2 # Vertical offset for hexagonal grid cells
x_min = theater_center_x - 2000
x_step = 200
y_min = theater_center_y - 2000 - (int(21/k)*k*200 - 4000)/2
y_step = 200 * k 

latitudes = []
longitudes = []
distances_from_center = []
xs = []
ys = []
for i in range(0, int(21/k)):
    y = y_min + i * y_step
    x_offset = 100 if i%2==0 else 0
    for j in range(0, 21):
        x = x_min + j * x_step + x_offset
        distance_from_center = calc_xy_distance(theater_center_x, theater_center_y, x, y)
        if (distance_from_center <= 2001):
            lon, lat = xy_to_lonlat(x, y)
            latitudes.append(lat)
            longitudes.append(lon)
            distances_from_center.append(distance_from_center)
            xs.append(x)
            ys.append(y)

print(len(latitudes), 'candidate neighborhood centers generated.')

364 candidate neighborhood centers generated.


Let's visualize the data we have so far: city center location and candidate neighborhood centers:

In [6]:
map_london = folium.Map(location=theater_center, zoom_start=13)
folium.Marker(theater_center, popup='Leicester Square').add_to(map_london)
for lat, lon in zip(latitudes, longitudes):
    #folium.CircleMarker([lat, lon], radius=2, color='blue', fill=True, fill_color='blue', fill_opacity=1).add_to(map_london) 
    folium.Circle([lat, lon], radius=100, color='blue', fill=False).add_to(map_london)
    #folium.Marker([lat, lon]).add_to(map_london)
map_london

OK, we now have the coordinates of centers of neighborhoods/areas to be evaluated, equally spaced (distance from every point to it's neighbors is exactly the same) and within ~1km from Leicester Square. 

Let's now use geolocator to get approximate addresses of those locations.

In [7]:
from geopy.geocoders import Nominatim 

def get_address(la, lo):
    try:
        geolocator = Nominatim(user_agent="address_explorer", timeout=200)
        location = geolocator.reverse(la, lo)
        address=location.address
        return address
    except:
        return None
addr = get_address(theater_center[0], theater_center[1])
print('Reverse geocoding check')
print('-----------------------')
print('Address of [{}, {}] is: {}'.format(theater_center[0], theater_center[1], addr))

Reverse geocoding check
-----------------------
Address of [51.5105992, -0.1308565] is: Aspen Way, Blackwall, London Borough of Tower Hamlets, London, Greater London, England, E14 9QS, United Kingdom


In [8]:
print('Obtaining location addresses: ', end='')
addresses = []
for lat, lon in zip(latitudes, longitudes):
    address = get_address(lat, lon)
    if address is None:
        address = 'NO ADDRESS'
    address = address.replace(', United Kingdom', '') # We don't need country part of address
    addresses.append(address)
    print(' .', end='')
print(' done.')

Obtaining location addresses:  . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . done.


In [9]:
 addresses[100:120]

['Arora Tower, 2, Blackwall Tunnel, Canary Wharf, Blackwall, London Borough of Tower Hamlets, London, Greater London, England, E14 9PQ',
 'Arora Tower, 2, Blackwall Tunnel, Canary Wharf, Blackwall, London Borough of Tower Hamlets, London, Greater London, England, E14 9PQ',
 'Arora Tower, 2, Blackwall Tunnel, Canary Wharf, Blackwall, London Borough of Tower Hamlets, London, Greater London, England, E14 9PQ',
 'Arora Tower, 2, Blackwall Tunnel, Canary Wharf, Blackwall, London Borough of Tower Hamlets, London, Greater London, England, E14 9PQ',
 'Arora Tower, 2, Blackwall Tunnel, Canary Wharf, Blackwall, London Borough of Tower Hamlets, London, Greater London, England, E14 9PQ',
 'Arora Tower, 2, Blackwall Tunnel, Canary Wharf, Blackwall, London Borough of Tower Hamlets, London, Greater London, England, E14 9PQ',
 'Arora Tower, 2, Blackwall Tunnel, Canary Wharf, Blackwall, London Borough of Tower Hamlets, London, Greater London, England, E14 9PQ',
 'Arora Tower, 2, Blackwall Tunnel, Canar

Looking good. Let's now place all this into a Pandas dataframe.

In [10]:

df_locations = pd.DataFrame({'Address': addresses,
                             'Latitude': latitudes,
                             'Longitude': longitudes,
                             'X': xs,
                             'Y': ys,
                             'Distance from center': distances_from_center})

df_locations.head(10)

,Address,Latitude,Longitude,X,Y,Distance from center
0,"GR40, Tunnel Avenue, Greenwich Peninsula, Roya...",51.493262,-0.138313,282153.38804,5.709351e+06,1997.498436
1,"GR40, Tunnel Avenue, Greenwich Peninsula, Roya...",51.493339,-0.135437,282353.38804,5.709351e+06,1946.792233
2,"GR40, Tunnel Avenue, Greenwich Peninsula, Roya...",51.493416,-0.132560,282553.38804,5.709351e+06,1915.724406
3,"GR40, Tunnel Avenue, Greenwich Peninsula, Roya...",51.493493,-0.129683,282753.38804,5.709351e+06,1905.255888
4,"GR40, Tunnel Avenue, Greenwich Peninsula, Roya...",51.493570,-0.126807,282953.38804,5.709351e+06,1915.724406
5,"GR40, Tunnel Avenue, Greenwich Peninsula, Roya...",51.493647,-0.123930,283153.38804,5.709351e+06,1946.792233
6,"GR40, Tunnel Avenue, Greenwich Peninsula, Roya...",51.493723,-0.121054,283353.38804,5.709351e+06,1997.498436
7,"GR40, Tunnel Avenue, Greenwich Peninsula, Roya...",51.494702,-0.142735,281853.38804,5.709524e+06,1951.922130
8,"GR40, Tunnel Avenue, Greenwich Peninsula, Roya...",51.494779,-0.139858,282053.38804,5.709524e+06,1868.154169
9,"GR40, Tunnel Avenue, Greenwich Peninsula, Roya...",51.494856,-0.136982,282253.38804,5.709524e+06,1802.775638


...and let's now save/persist this data into local file.

In [11]:
df_locations.to_pickle('./locations.pkl') 

### Foursquare
Now that we have our location candidates, let's use Foursquare API to get info on cafes in each neighborhood.

We're interested in venues in 'food' category, but only those that are proper cafes - pizza places, bakeries etc. are not direct competitors so we don't care about those. So we will include in out list only venues that have 'cafe' in category name.

In [12]:
food_category = '4d4b7105d754a06374d81259' # 'Root' category for all food-related venues

cafe_categories = ['4bf58dd8d48988d16d941735','4bf58dd8d48988d1e0931735'] # 'Root' category for café and coffee shop venues

def is_cafe(categories, specific_filter=None):
    cafe_words = ['café', 'coffee shop', 'coffee','cafe']
    cafe = False
    specific = False
    for c in categories:
        category_name = c[0].lower()
        category_id = c[1]
        for r in cafe_words:
            if r in category_name:
                cafe = True
                specific = True
        if category_id in specific_filter:
            specific = True
            cafe = True
    return cafe, specific

def get_categories(categories):
    return [(cat['name'], cat['id']) for cat in categories]

def format_address(location):
    address = ', '.join(location['formattedAddress'])
    address = address.replace(', United Kingdom', '')
    address = address.replace(', England', '')
    return address

client_id='H0RVNORB0NGOYAUCC2DAF24C0JS3CTYKXS5RPKEWM1TJXQWW'
client_secret='JTCXR4NA4TSVU5WMSO5RA1TO2LPJQOX2I3XDEIVSCBN2TCRS'
version = '20180605'

def get_venues_near_location(lat, lon, category, client_id, client_secret, radius=100, limit=100):
    url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&categoryId={}&radius={}&limit={}'.format(
        client_id, client_secret, version, lat, lon, category, radius, limit)
    try:
        results = requests.get(url).json()['response']['groups'][0]['items']
        venues = [(item['venue']['id'],
                   item['venue']['name'],
                   get_categories(item['venue']['categories']),
                   (item['venue']['location']['lat'], item['venue']['location']['lng']),
                   format_address(item['venue']['location']),
                   item['venue']['location']['distance']) for item in results]        
    except:
        venues = []
    return venues

In [13]:
# Let's now go over our neighborhood locations and get nearby cafes.
import pickle

def get_cafes(lats, lons):
    cafes = {}
    location_cafes = []

    print('Obtaining venues around candidate locations:', end='')
    for lat, lon in zip(lats, lons):
        # Using radius=150 to meke sure we have overlaps/full coverage so we don't miss any cafe (we're using dictionaries to remove any duplicates resulting from area overlaps)
        venues = get_venues_near_location(lat, lon, food_category, client_id, client_secret, radius=150, limit=100)
        area_cafes = []
        for venue in venues:
            venue_id = venue[0]
            venue_name = venue[1]
            venue_categories = venue[2]
            venue_latlon = venue[3]
            venue_address = venue[4]
            venue_distance = venue[5]
            is_ca,is_co = is_cafe(venue_categories, specific_filter=cafe_categories)
            if is_ca:
                x, y = lonlat_to_xy(venue_latlon[1], venue_latlon[0])
                cafe = (venue_id, venue_name, venue_latlon[0], venue_latlon[1], venue_address, venue_distance, is_co, x, y)
                if venue_distance<=100:
                    area_cafes.append(cafe)
                cafes[venue_id] = cafe
        location_cafes.append(area_cafes)
        print(' .', end='')
    print(' done.')
    return cafes, location_cafes

# Try to load from local file system in case we did this before
cafes = {}
location_cafes = []
loaded = False
try:
    with open('cafes_150.pkl', 'rb') as f:
        cafes = pickle.load(f)
    with open('location_cafes_150.pkl', 'rb') as f:
        location_cafes = pickle.load(f)
    print('Cafe data loaded.')
    loaded = True
except:
    pass

# If load failed use the Foursquare API to get the data
if not loaded:
    cafes, location_cafes = get_cafes(latitudes, longitudes)
    
    # Let's persists this in local file system
    with open('cafes_150.pkl', 'wb') as f:
        pickle.dump(cafes, f)
    with open('location_cafes_150.pkl', 'wb') as f:
        pickle.dump(location_cafes, f)

Cafe data loaded.


In [14]:
# Statistics
print('Total number of cafes:', len(cafes))
print('Average number of cafets in neighborhood:', np.array([len(r) for r in location_cafes]).mean())

Total number of cafes: 198
Average number of cafets in neighborhood: 0.42032967032967034


In [15]:
print('List of all cafes')
print('-----------------------')
for r in list(cafes.values())[:10]:
    print(r)
print('...')
print('Total:', len(cafes))

List of all cafes
-----------------------
('4ae2b056f964a5204f8f21e3', 'Regency Cafe', 51.49389527652026, -0.13231140771600738, '17-19 Regency St. (near Horseferry Rd.), City of Westminster, Greater London, SW1P 4BY', 145, True, 282572.9202197246, 5709403.168359492)
('4d5f61db5b276dcb357807c6', 'Astral Cafe', 51.494701158344704, -0.13241527788843818, '8 Regency Pl (Horseferry Rd), City of Westminster, Greater London, SW1P 2EB', 89, True, 282569.54868514126, 5709493.071697018)
('4bb495885148d13a67ac523b', 'Sapori Café & Restaurant', 51.49480549750268, -0.12765355783368967, '60 Horseferry Rd, London, Greater London, SW1P 2AF', 57, True, 282900.4963106291, 5709490.530700196)
('4c7264dcad69b60c7e4483b9', "Gianni's", 51.49486063887886, -0.12713937109640466, '48 Horseferry Rd, London, Greater London, SW1P 2AF', 120, True, 282936.4416335095, 5709495.135395427)
('4dd2ab00fa767fb7079834f3', 'Burberry Cafétaria', 51.49470119499509, -0.1265488224820957, 'Horseferry House, London, Greater London',

In [16]:
print('Cafes around location')
print('---------------------------')
for i in range(20, 40):
    rs = location_cafes[i][:8]
    names = ', '.join([r[1] for r in rs])
    print('Cafes around location {}: {}'.format(i+1, names))

Cafes around location
---------------------------
Cafes around location 21: 
Cafes around location 22: XLNT, Cafe Fresco
Cafes around location 23: 
Cafes around location 24: The Wash House Cafe
Cafes around location 25: 
Cafes around location 26: 
Cafes around location 27: 
Cafes around location 28: 
Cafes around location 29: 
Cafes around location 30: 
Cafes around location 31: 
Cafes around location 32: 
Cafes around location 33: The Rail House Cafe
Cafes around location 34: 
Cafes around location 35: 
Cafes around location 36: 
Cafes around location 37: 
Cafes around location 38: Cellarium Cafe & Terrace
Cafes around location 39: 
Cafes around location 40: 


In [17]:
map_london = folium.Map(location=theater_center, zoom_start=13)
folium.Marker(theater_center, popup='Leicester Square').add_to(map_london)
for cas in cafes.values():
    lat = cas[2]; lon = cas[3]
    color = 'blue' 
    folium.CircleMarker([lat, lon], radius=1, color=color, fill=True, fill_color=color, fill_opacity=1).add_to(map_london)
map_london

Looking good. So now we have all the cafes in area within 1 kilometer from Leicester Square! We also know which cafes exactly are in vicinity of every neighborhood candidate center.

This concludes the data gathering phase - we're now ready to use this data for analysis to produce the report on optimal locations for a new musical theme cafe!